In [ ]:
!git clone https://github.com/Tianxiaomo/pytorch-YOLOv4.git

Cloning into 'pytorch-YOLOv4'...
remote: Enumerating objects: 1049, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1049 (delta 2), reused 0 (delta 0), pack-reused 1043
Receiving objects: 100% (1049/1049), 2.39 MiB | 8.68 MiB/s, done.
Resolving deltas: 100% (644/644), done.


In [ ]:
!pwd
!ls

/content
pytorch-YOLOv4	sample_data


In [ ]:
import os
os.chdir("pytorch-YOLOv4")

In [ ]:
from tool.darknet2pytorch import Darknet
import torch
import cv2

In [ ]:
m = Darknet('cfg/yolov4-tiny.cfg')
m.load_weights('yolov4-tiny.weights')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
img = cv2.imread('data/dog.jpg')
sized = cv2.resize(img, (m.width, m.height))
sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)
img = torch.from_numpy(sized.transpose(2, 0, 1)).float().div(255.0).unsqueeze(0)

In [ ]:
first_block = m.models[0]
first_block
second_block = m.models[1]
second_block

Sequential(
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky2): LeakyReLU(negative_slope=0.1, inplace=True)
)

In [ ]:
def save_to_file(path, data):
    print (f"Writing data with shape {data.shape} to file {path}")
    with open(path, "wb") as f:
        data.tofile(f)

In [ ]:
def fuse_conv_and_bn(conv, bn):
#
# init
    fusedconv = torch.nn.Conv2d(
    conv.in_channels,
    conv.out_channels,
    kernel_size=conv.kernel_size,
    stride=conv.stride,
    padding=conv.padding,
    bias=True
    )
    #
    # prepare filters
    w_conv = conv.weight.clone().view(conv.out_channels, -1)
    w_bn = torch.diag(bn.weight.div(torch.sqrt(bn.eps+bn.running_var)))
    #fusedconv.weight.copy_( torch.mm(w_bn, w_conv).view(fusedconv.weight.size()) )
    new_weight = torch.mm(w_bn, w_conv).view(fusedconv.weight.size())
    fusedconv.weight = torch.nn.Parameter(new_weight)
    #
    # prepare spatial bias
    if conv.bias is not None:
        b_conv = conv.bias
    else:
        b_conv = torch.zeros( conv.weight.size(0) )
    b_bn = bn.bias - bn.weight.mul(bn.running_mean).div(torch.sqrt(bn.running_var + bn.eps))
    #fusedconv.bias.copy_( torch.matmul(w_bn, b_conv) + b_bn )
    new_bias = torch.matmul(w_bn, b_conv) + b_bn
    fusedconv.bias = torch.nn.Parameter(new_bias)
    #
    # we're done
    return fusedconv

In [ ]:
conv_layer1 = first_block[0]
bn1 = first_block[1]
leaky1 = first_block[2]
bn1.bias
conv_layer2 = second_block[0]
bn2 = second_block[1]
leaky2 = second_block[2]

In [ ]:
batchnorm_weight_name = './bin/batchnorm_weight.bin'
batchnorm_layer = first_block[1]
save_to_file(batchnorm_weight_name, batchnorm_layer.weight.data.numpy())

Writing data with shape (32,) to file ./bin/batchnorm_weight.bin


In [ ]:
batchnorm_bias_name = './bin/batchnorm_bias.bin'
batchnorm_layer = first_block[1]
save_to_file(batchnorm_bias_name, batchnorm_layer.bias.data.numpy())

Writing data with shape (32,) to file ./bin/batchnorm_bias.bin


In [ ]:
EPS = bn1.eps
bn_factor    = torch.div(bn1.weight,torch.sqrt(bn1.running_var+EPS)).view(-1,1,1,1)
fused_weight = torch.mul(conv_layer1.weight, bn_factor)
fused_bias   = bn1.bias - torch.div(torch.mul(bn1.weight,bn1.running_mean),torch.sqrt(bn1.running_var+EPS))

In [ ]:
fused_bias_name = '../fused_bias.bin'
bias_l1 = fused_bias
save_to_file(fused_bias_name, bias_l1.data.numpy())

fused_weight_name = '../fused_weight.bin'
weight_l1 = fused_weight
save_to_file(fused_weight_name, weight_l1.data.numpy())

Writing data with shape (32,) to file ../fused_bias.bin
Writing data with shape (32, 3, 3, 3) to file ../fused_weight.bin


In [ ]:
fused_conv = fuse_conv_and_bn(conv_layer1, bn1) 
fused_conv

Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))

In [ ]:
conv_layer_bias_name = '../conv_bias.bin'
bias_l1 = fused_conv.bias
save_to_file(conv_layer_bias_name, bias_l1.data.numpy())

Writing data with shape (32,) to file ../conv_bias.bin


In [ ]:
conv_layer_mod_wts_name = '../conv_weight_bn.bin'
conv_l1_wts_bn = fused_conv.weight
save_to_file(conv_layer_mod_wts_name, conv_l1_wts_bn.data.numpy())

Writing data with shape (32, 3, 3, 3) to file ../conv_weight_bn.bin


In [ ]:
#Original output
conv_layer = first_block[0]
conv_output = conv_layer(img)
batchnorm_layer = first_block[1]
batchnorm_output = batchnorm_layer(conv_output)

In [ ]:
#Fused conv output
fused_output = fused_conv(img)

In [ ]:
net = torch.nn.Sequential(conv_layer1,bn1)
y1 = net.forward(img)
y2 = fused_conv.forward(img)
d = (y1 - y2).norm().div(y1.norm()).item()
print("error: %.8f" % d)

error: 0.19199745


# **Second Layer Parameters Extraction**

In [ ]:
second_block = m.models[1]
second_block

Sequential(
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky2): LeakyReLU(negative_slope=0.1, inplace=True)
)

In [ ]:
conv_layer2 = second_block[0]
bn2 = second_block[1]
leaky2 = second_block[2]

In [ ]:
conv_layer_weight_name2 = './bin/conv_weight2.bin'
save_to_file(conv_layer_weight_name2, conv_layer2.weight.data.numpy())

Writing data with shape (64, 32, 3, 3) to file ./bin/conv_weight2.bin


Output of first layer is input to second

In [ ]:
out1_conv = conv_layer1(img)
out1_batch = bn1(out1_conv)
output_layer1 = leaky1(out1_batch)

In [ ]:
# write the array to a binary file
input_to_second_name = './bin/conv_input2.bin'
save_to_file(input_to_second_name, output_layer1[0].detach().numpy())

Writing data with shape (32, 208, 208) to file ./bin/conv_input2.bin


In [ ]:
batchnorm_weight_name2 = './bin/batchnorm_weight2.bin'

save_to_file(batchnorm_weight_name2, bn2.weight.data.numpy())

Writing data with shape (64,) to file ./bin/batchnorm_weight2.bin


In [ ]:
batchnorm_bias_name2 = './bin/batchnorm_bias2.bin'

save_to_file(batchnorm_bias_name2, bn2.bias.data.numpy())

Writing data with shape (64,) to file ./bin/batchnorm_bias2.bin


In [ ]:
conv_output2 = conv_layer2(output_layer1)
batchnorm_output2 = bn2(conv_output2)
output_layer2 = leaky2(batchnorm_output2)

In [ ]:
conv_layer_output_name2 = './bin/conv_output2.bin'

save_to_file(conv_layer_output_name2, conv_output2[0].detach().numpy())

Writing data with shape (64, 104, 104) to file ./bin/conv_output2.bin


In [ ]:
batchnorm_layer_output_name2 = './bin/batchnorm_output2.bin'

save_to_file(batchnorm_layer_output_name2, batchnorm_output2[0].detach().numpy())

Writing data with shape (64, 104, 104) to file ./bin/batchnorm_output2.bin


In [ ]:
relu_layer_output_name2 = './bin/relu_output2.bin'

save_to_file(relu_layer_output_name2, output_layer2[0].detach().numpy())

Writing data with shape (64, 104, 104) to file ./bin/relu_output2.bin


TRYING OUT FUSING FROM SIBI

In [ ]:
import re
write_layer_outputs_to_file = True
write_layer_params_to_file  = True

features = {}
def get_features(name):
  def hook(model, input, output):
    features[name] = output.detach()
  return hook

model = m.models[0] #Change the model numbers here. This is the first layer now
model.eval()
print(model)

inp = torch.ones(1, 3, 736, 1280)

raw_layers = []
for layer in model.named_modules():
  raw_layers.append(layer[0])

leaf_layers = []
for i in range(1, len(raw_layers)-1):
  curr_layer = raw_layers[i]
  next_layer = raw_layers[i+1]
  if(next_layer[:len(curr_layer)+1] != curr_layer + "."):
    leaf_layers.append(curr_layer)
leaf_layers.append(next_layer)

layers = []
for i in range(len(leaf_layers)):
  layers.append(re.sub(r"\.(\d)",r"[\1]",leaf_layers[i]))

for i in range(len(layers)):
  layer = layers[i]
  layer_hook = "model." + layer + ".register_forward_hook(get_features('" + layer + "'))"
  exec(layer_hook)

# Run inference
outp = model(inp)

EPS = 10 ** -5 # constant

if(write_layer_outputs_to_file):
  # Write layer outputs
  for i in range(len(layers)):
    layer = layers[i]
    if(layer in features.keys()):
      layer_name = layer.replace("].","_")
      layer_name = layer_name.replace("[", "_")
      layer_name = layer_name.replace("]", "")
      filename = "../" +  layer_name + ".bin"
      with open(filename,"wb") as f:
        features[layer].cpu().numpy().tofile(f)
      print("Layer " + str(i) + " feature map printed to " + filename)

if(write_layer_params_to_file):
  # Write layer params
  for i in range(len(layers)):
    layer = layers[i]
    if('conv' in layer or 'downsample[0]' in layer):
      conv_layer_name = layer.replace("].","_")
      conv_layer_name = conv_layer_name.replace("[", "_")
      conv_layer_name = conv_layer_name.replace("]", "")
      
      conv_param_name = layer.replace("[",".")
      conv_param_name = conv_param_name.replace("]","")
      
      conv_weight = model.state_dict()[conv_param_name+'.weight']

    if('bn' in layer or 'downsample[1]' in layer):
      bn_layer_name = layer.replace("].","_")
      bn_layer_name = bn_layer_name.replace("[", "_")
      bn_layer_name = bn_layer_name.replace("]", "")
      
      bn_param_name = layer.replace("[",".")
      bn_param_name = bn_param_name.replace("]","")
      
      bn_weight = model.state_dict()[bn_param_name+'.weight']
      bn_bias   = model.state_dict()[bn_param_name+'.bias']
      bn_mean   = model.state_dict()[bn_param_name+'.running_mean']
      bn_var    = model.state_dict()[bn_param_name+'.running_var']

      bn_factor    = torch.div(bn_weight,torch.sqrt(bn_var+EPS)).view(-1,1,1,1)
      fused_weight = torch.mul(conv_weight, bn_factor)
      fused_bias   = bn_bias - torch.div(torch.mul(bn_weight,bn_mean),torch.sqrt(bn_var+EPS))

      if('downsample' in bn_layer_name):
        layer_number = '0'
        layer_prefix = bn_layer_name[0:bn_layer_name.find('downsample')]
      else:
        layer_number = conv_layer_name[-1]
        layer_prefix = bn_layer_name[0:bn_layer_name.find('bn')]
      
      weights_filename = "../fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_weights.bin" #This gives the bin for weights
      bias_filename    = "../fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_bias.bin" #This gives the bin for bias

      with open(weights_filename, "wb") as f:
        fused_weight.detach().numpy().tofile(f)
      print("Fused weights of " + layer_prefix + layer_number + " printed to file " + weights_filename)

      with open(bias_filename, "wb") as f:
        fused_bias.detach().numpy().tofile(f)
      print("Fused biases of " + layer_prefix + layer_number + " printed to file " + bias_filename)

Sequential(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky1): LeakyReLU(negative_slope=0.1, inplace=True)
)
Layer 0 feature map printed to ../conv1.bin
Layer 1 feature map printed to ../bn1.bin
Layer 2 feature map printed to ../leaky1.bin
Fused weights of 1 printed to file ../fused_conv1_bn1_weights.bin
Fused biases of 1 printed to file ../fused_conv1_bn1_bias.bin
